In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM, CuDNNLSTM
from keras.layers import Conv1D, Flatten
from keras.datasets import imdb
import wandb
from wandb.keras import WandbCallback
import numpy as np
from keras.preprocessing import text

wandb.init()
config = wandb.config

Using TensorFlow backend.


W&B Run: https://app.wandb.ai/univai-ss2019/CaptionHack/runs/gl5qv1br
Call `%%wandb` in the cell containing your training loop to display live results.


In [2]:
from keras import __version__
from keras.applications.vgg16 import VGG16, preprocess_input

In [3]:
from keras.models import Sequential
from keras.layers import Flatten, Activation, Dense

In [4]:
from keras.preprocessing.image import ImageDataGenerator


In [5]:
train_txt = "../flickr/Flickr_8k.trainImages.txt"
val_txt = "../flickr/Flickr_8k.devImages.txt"
test_txt = "../flickr/Flickr_8k.testImages.txt"
token_txt = "../flickr/Flickr8k.token.txt" 

In [6]:
data_dir = "../flickr/datasets"

In [7]:
captions = [e.strip().split("\t") for e in open(token_txt).readlines()]
trainfiles = [e.strip() for e in open(train_txt).readlines()]
devfiles = [e.strip() for e in open(val_txt).readlines()]
testfiles = [e.strip() for e in open(test_txt).readlines()]

In [8]:
captions[0], trainfiles[0], devfiles[0], testfiles[0]

(['1000268201_693b08cb0e.jpg#0',
  'A child in a pink dress is climbing up a set of stairs in an entry way .'],
 '2513260012_03d33305cf.jpg',
 '2090545563_a4e66ec76b.jpg',
 '3385593926_d3e9c21170.jpg')

In [9]:
# datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# generator = datagen.flow_from_directory(
#         data_dir, class_mode="categorical")

In [10]:
from collections import defaultdict
captiondict = defaultdict(list)
for pair in captions:
    image = pair[0].split('#')[0]
    caption = pair[1]
    captiondict[image].append(caption)
traindict = {}
devdict = {}
testdict = {}
for f in trainfiles:
    traindict[f] = captiondict[f]
for f in devfiles:
    devdict[f] = captiondict[f]
for f in testfiles:
    testdict[f] = captiondict[f]

In [11]:
from keras.preprocessing import text

In [12]:
tokenizer = text.Tokenizer()

In [13]:
captions_train = [captiondict[f] for f in trainfiles]
captions_valid = [captiondict[f] for f in devfiles]
captions_test = [captiondict[f] for f in testfiles]

In [14]:
from  itertools import chain
train_list = list(chain.from_iterable(traindict.values()))
validation_list = list(chain.from_iterable(devdict.values()))
test_list = list(chain.from_iterable(testdict.values()))

In [15]:
tokenizer.fit_on_texts(train_list)
X_train = tokenizer.texts_to_sequences(train_list)
X_valid = tokenizer.texts_to_sequences(validation_list)
X_test = tokenizer.texts_to_sequences(test_list)

In [16]:
num_classes = len(tokenizer.word_index) +1

In [17]:
index_word_dict = tokenizer.index_word
word_index_dict = tokenizer.word_index

In [18]:
word = []
next_word = []
for list_t in X_train:
    for i in range(1,len(list_t)):
        word.append(list_t[i-1])
        next_word.append(list_t[i])
            

In [19]:
word_val = []
next_word_val = []
for list_t in X_valid:
    for i in range(1,len(list_t)):
        word_val.append(list_t[i-1])
        next_word_val.append(list_t[i])

In [20]:
len(word), len(next_word)

(294479, 294479)

In [21]:
len(word_val), len(next_word_val)

(48962, 48962)

In [22]:
max(next_word_val), num_classes

(7371, 7376)

In [23]:
from keras.utils import to_categorical


In [24]:
X_train_f= to_categorical(word, num_classes= num_classes)

In [25]:
y_train_f = to_categorical(next_word, num_classes = num_classes)

In [26]:
X_valid_f = to_categorical(word_val, num_classes= num_classes)


In [27]:
y_valid_f = to_categorical(next_word_val, num_classes= num_classes)

In [28]:
X_train_f.shape

(294479, 7376)

In [31]:
len(word)

294479

In [ ]:
model = Sequential()
model.add(Embedding(num_classes, 50, input_length = len(word)))
model.add(CuDNNLSTM(50))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop')

Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:38: UserWarning: [Errno 12] Cannot allocate memory.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [ ]:
model.fit(X_train_f, y_train_f,
          batch_size=64,
          epochs=10,
          validation_data=(X_valid_f, y_valid_f), callbacks=[WandbCallback()])